In [11]:
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os

In [12]:
os.getcwd()

'/Users/faraz.bukhari/Documents/MyCodes/ML'

In [165]:
df=pd.read_csv('/Users/faraz.bukhari/Downloads/ReviewsClassifier.csv')
df.fillna(0,inplace=True)
df.head()

,text,low_price,no_response,poor_service,satisfied
0,Overall experience was good,0.0,0.0,0.0,1.0
1,Very Professional & Quick In Evaluation,0.0,0.0,0.0,1.0
2,"Other than pricing, everything else is fine.",1.0,0.0,0.0,0.0
3,Not got compititive price,1.0,0.0,0.0,0.0
4,Very poor customer communication. OLX represen...,0.0,0.0,1.0,0.0


# Text Preprocessing

In [166]:
#check nulls
df.isnull().sum()

text            0
low_price       0
no_response     0
poor_service    0
satisfied       0
dtype: int64

In [167]:
#check duplicates
df.duplicated().sum()

29

In [168]:
#clean text
def clean_text(text):
    text = str(text)
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [169]:
df['text'] = df['text'].apply(clean_text)
df.head()

,text,low_price,no_response,poor_service,satisfied
0,overall experience was good,0.0,0.0,0.0,1.0
1,very professional quick in evaluation,0.0,0.0,0.0,1.0
2,other than pricing everything else is fine,1.0,0.0,0.0,0.0
3,not got compititive price,1.0,0.0,0.0,0.0
4,very poor customer communication olx represent...,0.0,0.0,1.0,0.0


In [170]:
import nltk
nltk.download()
#nltk.download('punkt')
def transform_text(text):
    #lowercase
    text=text.lower()
    #tokenization
    text = nltk.word_tokenize(text)
    #removing special characters
    y = []
    for i in text:
        if i.isalnum():
            y.append(i)
    #removing stopwords and punctuations
    from nltk.corpus import stopwords
    import string
    text = y[:]
    y.clear()
    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)
    #stemming
    from nltk.stem.porter import PorterStemmer
    ps = PorterStemmer()
    import string
    text = y[:]
    y.clear()
    for i in text:
        y.append(ps.stem(i))

    return " ".join(y)

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [171]:
transform_text('My name is the a faraz you !, lovingly')

'name faraz lovingli'

In [172]:
df.head()

,text,low_price,no_response,poor_service,satisfied
0,overall experience was good,0.0,0.0,0.0,1.0
1,very professional quick in evaluation,0.0,0.0,0.0,1.0
2,other than pricing everything else is fine,1.0,0.0,0.0,0.0
3,not got compititive price,1.0,0.0,0.0,0.0
4,very poor customer communication olx represent...,0.0,0.0,1.0,0.0


In [173]:
df['transformed_text'] = df['text'].apply(transform_text)
df.head()

,text,low_price,no_response,poor_service,satisfied,transformed_text
0,overall experience was good,0.0,0.0,0.0,1.0,overal experi good
1,very professional quick in evaluation,0.0,0.0,0.0,1.0,profession quick evalu
2,other than pricing everything else is fine,1.0,0.0,0.0,0.0,price everyth els fine
3,not got compititive price,1.0,0.0,0.0,0.0,got compitit price
4,very poor customer communication olx represent...,0.0,0.0,1.0,0.0,poor custom commun olx repres train speak custom


# Feature Engineering

In [181]:
df_labelled = df[:178]
df_unlabelled = df[179:]
df_labelled.tail()

,text,low_price,no_response,poor_service,satisfied,transformed_text
173,rates quoted was much lower than market,1.0,0.0,0.0,0.0,rate quot much lower market
174,the price wasn t offered till date even after ...,0.0,1.0,0.0,0.0,price offer till date even multipl follow up r...
175,not provided best price as you showing on adve...,1.0,0.0,0.0,0.0,provid best price show advertis
176,you don t work as per your motto in your adver...,1.0,0.0,0.0,0.0,work per motto advertis say give best price gi...
177,i do not get satisfactory price of my vehicle,1.0,0.0,0.0,0.0,get satisfactori price vehicl


In [182]:
df_unlabelled.head()

,text,low_price,no_response,poor_service,satisfied,transformed_text
179,give me a best price,0.0,0.0,0.0,0.0,give best price
180,i was told that olx offers best price but very...,0.0,0.0,0.0,0.0,told olx offer best price meagr price offer sa...
181,u r not give any rate for my old car,0.0,0.0,0.0,0.0,u r give rate old car
182,quote and actual offer is much in variance not...,0.0,0.0,0.0,0.0,quot actual offer much varianc even close
183,had to call several times to get valuation olx...,0.0,0.0,0.0,0.0,call sever time get valuat olx interest buy ve...


In [185]:
corpus = df_labelled['transformed_text']
print(type(corpus))
corpus.head()

<class 'pandas.core.series.Series'>


0                                  overal experi good
1                              profession quick evalu
2                              price everyth els fine
3                                  got compitit price
4    poor custom commun olx repres train speak custom
Name: transformed_text, dtype: object

In [186]:
tfidf = TfidfVectorizer()
Xfeatures = tfidf.fit_transform(corpus).toarray()

In [187]:
Xfeatures[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [193]:
df.columns

Index(['text', 'low_price', 'no_response', 'poor_service', 'satisfied',
       'transformed_text'],
      dtype='object')

In [194]:
categories=['low_price', 'no_response', 'poor_service', 'satisfied']

In [197]:
y = df_labelled[categories]
y

,low_price,no_response,poor_service,satisfied
0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0
...,...,...,...,...
173,1.0,0.0,0.0,0.0
174,0.0,1.0,0.0,0.0
175,1.0,0.0,0.0,0.0
176,1.0,0.0,0.0,0.0


# Modelling

In [198]:
X_train,X_test,y_train,y_test = train_test_split(Xfeatures,y,test_size=0.3,random_state=42)

In [ ]:
#Binary relevance technique is used to convert our multi-label problem to a multi-class problem.

In [199]:
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset

In [200]:
binary_rel_clf = BinaryRelevance(MultinomialNB())

In [201]:
binary_rel_clf.fit(X_train,y_train)

BinaryRelevance(classifier=MultinomialNB(), require_dense=[True, True])

In [202]:
BinaryRelevance(classifier=MultinomialNB(alpha=1.0, class_prior=None,
                                         fit_prior=True),
                require_dense=[True, True])

BinaryRelevance(classifier=MultinomialNB(), require_dense=[True, True])

In [203]:
#prediction

In [204]:
br_prediction = binary_rel_clf.predict(X_test)

In [205]:
br_prediction.toarray()

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 0.],


In [206]:
accuracy_score(y_test,br_prediction)

0.2962962962962963

In [207]:
from sklearn.metrics import accuracy_score,hamming_loss
hamming_loss(y_test,br_prediction)

0.18981481481481483

# Predictions

In [208]:
df_unlabelled.head()

,text,low_price,no_response,poor_service,satisfied,transformed_text
179,give me a best price,0.0,0.0,0.0,0.0,give best price
180,i was told that olx offers best price but very...,0.0,0.0,0.0,0.0,told olx offer best price meagr price offer sa...
181,u r not give any rate for my old car,0.0,0.0,0.0,0.0,u r give rate old car
182,quote and actual offer is much in variance not...,0.0,0.0,0.0,0.0,quot actual offer much varianc even close
183,had to call several times to get valuation olx...,0.0,0.0,0.0,0.0,call sever time get valuat olx interest buy ve...


In [211]:
df_new=df_unlabelled[['transformed_text']]

In [212]:
df_new.head()

,transformed_text
179,give best price
180,told olx offer best price meagr price offer sa...
181,u r give rate old car
182,quot actual offer much varianc even close
183,call sever time get valuat olx interest buy ve...


In [98]:
#making a single prediction

In [220]:
ex1 = df_new['transformed_text'].iloc[200]

In [221]:
vec_example = tfidf.transform([ex1])

In [222]:
ans=binary_rel_clf.predict(vec_example).toarray()
print(ex1)
print(ans)
print(get_labels(ans[0]))

chief poor
[[0. 0. 0. 0.]]
None


In [223]:
ex1

'chief poor'

In [224]:
def get_labels(arr):
    i=0
    for a in arr:
        if int(a)!=0:
            print(categories[i])
        i=i+1

In [228]:
for r in df_new['transformed_text']:
    vec_example = tfidf.transform([r])
    ans=binary_rel_clf.predict(vec_example).toarray()
    #print(ans)
    print(r)
    i=0
    for a in ans[0]:
        if int(a)!=0:
            print(categories[i])
        i=i+1
    print('-----------')
    

give best price
low_price
-----------
told olx offer best price meagr price offer sale car excel work condit
low_price
-----------
u r give rate old car
low_price
-----------
quot actual offer much varianc even close
low_price
-----------
call sever time get valuat olx interest buy vehicl
-----------
inspect valu car least
low_price
-----------
poor price provid price provid rs 45 000 less competitor
low_price
-----------
becouus 1 tym convers
-----------
one take respons revert mani followup still receiv offer side alway said take 24 hr 24 hr come still think worst platform
no_response
-----------
inspect commun olx auto good howev could meet target patric expect therefor car deal happen
-----------
nobodi contact post inspect suddenli get regret messag
no_response
-----------
sincer
-----------
inspect complet silenc olx day even follow telephon time still respons
no_response
-----------
updat price one one servic
-----------
car price low everyday defer person call ask question
----

fake auction fake price
-----------
execut respond properli even conclud
-----------
unprofession set made falsehood untrain staff get hold get bad name entir iim bengaluru commun never touch olx futur
-----------
car inspect 2a thorough accur howev valu car highli assess
-----------
lot reason
-----------
good prompt servic
-----------
less price offer
low_price
-----------
guy said receiv cal back receiv singl call incept stop say olx auto deta hai car ka best price sound hilari u guy even provid singl offer
-----------
least price given competitor given way better price
low_price
-----------
valuat poor much beyond actual market valu
low_price
-----------
visit
-----------
inspect took around 2 hr time auction gad done next day
-----------
guy show adverti pay best sell price got total wast valuabl time everi inform share websit book inspect slot quot minimum 45 rest base inspect got offer kabadi wali cud pay offer show websit
-----------
worst car valu 6 month old car
-----------
g

inspect mathod appropri need experi vehicl like owner dose like sale person interest favour client inspect done gain maximum custom favour olx
-----------
rate issu
-----------
offer price low
low_price
-----------
given lowest price industri
low_price
-----------
inspect commun
-----------
fix car rate low
low_price
-----------
took long inspect quot price also price quot way other offer
low_price
-----------
took age schedul visit almost month post visit noth happen
-----------
much call extrem delay
-----------
nobodi contact inspect offer rate
-----------
2 week ho gye inspect hue aaj tk koi updat nahi h ke kya rate aaya gadi ka
no_response
-----------
reason car valu less get high price market
low_price
-----------
vehicl quot directli lakh rupe agenc market
low_price
-----------
noth exect
-----------
give good rate car
-----------
price quot less best price
low_price
-----------
person came inspect vehicl inform us submit report vehicl put site sale view india inspect receiv pho

price offer lesser compar cars24
low_price
-----------
disappoint quot extrem low price even 30 price lower car 24 spinni quot much better olx seem fraud kind compani olx
low_price
-----------
bad technician peopl
low_price
-----------
person experi peopl undervalu car other
-----------
offer least valu competitor
low_price
-----------
olx ask low price
low_price
-----------
opt less price car
low_price
-----------
smount quot car
low_price
-----------
user friendli
-----------
repli olx auto last one week
-----------
low price quot compar site
low_price
-----------
low valuat car averag respons team
-----------
respons incept
-----------
everyth good except price
-----------
delay price 3 day provid price price much low shown inspect inspect price vehicl good condit valu rupe 9 50 lac 10 80 around execut told inspect car condit excel scratch less valu given 7 70 lac continu call custom care show price inspect said wait 24 hour 3 day continu call provid worst experi
low_price
---------

much low price offer post negoti start
low_price
-----------
explain thing detail honestli feel satisfact inspect staff inspect guidanc fruit
-----------
price lowest price olx
low_price
-----------
responc done immedi
-----------
inspect custom updat fine trustabl even final quotat start paper work sale execut cancel deal worst
-----------
happi deal
low_price
-----------
need book inspect multipl time one came inspect
-----------
bogu inspector highli unprofession lot attitud even courtesi close car door park back place inspect done secondli got quot car use wast 45 minut get inspect done
-----------
price estim olx offer 55k less car sold outsid ooooop bad experi
low_price
-----------
nonsens servic
-----------
price poor better price reciv road side car shop
-----------
car valu less coordin
low_price
-----------
hopeless rate given car
-----------
liar think custom fool understand gve wrong price fill firm u drastic reduc half amount end
low_price
-----------
total confus staff la

...

mani discrep
-----------
വണ ട വല ഇട ന നത ൽ വളര മ ശ കച ചവടക ക ർ പ ല ഇത ല നന ന യ നല ല വ ലയ ട
-----------
low quot comparison other
low_price
-----------
worst tri fool custom give 50 percentag amount
low_price
-----------
coordin form team
no_response
-----------
nobodi contact inspect car irrespect coupl complaint unprofession compani
no_response
-----------
olx auto give price car 2 lac less current market price want sell grand i10 sportz 5 yr old got price 195000 see compani app olx find peopl sell 6 7 yr old grand i10 4 5 lac believ guy quot price 5 yr old grand i10 1 95 000 expect 4 lac guy readi 3 lac gave less price understand unaccept never recommend olx auto
low_price
-----------
highli impract stereotyp assess
-----------
give price half car price
low_price
-----------
highli unsatisfactori
low_price
-----------
could satisfi olx auto get proper coordin rate car 8 10 day got good offer neither get call compani discuss talk vehicl sorri worst experi olx
-----------
bad experi co

cote vert less price high end car
low_price
-----------
slow respons spinni much better respond day vs olx respond 3 4 day also price quot 20 lower spinni price
-----------
genuin
-----------
worst servic
-----------
commun defin deadlin dealin extend three time felt sad
-----------
overal servic good demerit poor coordin servic engin assign inspect first person assign job inspect rude turn time decid never bother inform subsequ inspect done next day disappoint servic begin merit fair price offer demerit express desir better price commun frozen respons team forc sell car anoth buyer
-----------
process system
-----------
price commit onlin actual offer differ lot despit good condit
low_price
-----------
respons team inspect
no_response
-----------
sold car 2 year ago olx car still transfer still get park speed ticket
no_response
-----------
15 day sinc valut done till date fair price car told bad
-----------
offer price three time lower actual sale price home inspect servic
-----------

follow olx better rate advert regard best rate etc market tactic rate offer way par dealer call regard ad put offer much olx
low_price
-----------
got price got competit henc recommend comparison
low_price
-----------
price gap portal
low_price
-----------
one contact inspect car
no_response
-----------
bad commit worest price
low_price
-----------
suppos get anoth call reconsid price execut call ask call next 20 min didnt anoth call
-----------
good servic around
-----------
get gud price car low
low_price
-----------
market price car around 2 lakh peopl underestim valu car u r quot 80000 complet wast time peopl car tire cost come 40 000 rs continent tire
low_price
-----------
car valu evalu proper
-----------
price inspect less cars24 com far far better
-----------
slow process irrespons peopl time scene
-----------
never got revert olx sell price
-----------
respons post inspect
-----------
bed
-----------
inspect nobodi side bother inform final result evalu inform regard offer pric

mani folloup incept guy came yet receiv quotat guy wast time
no_response
-----------
despit promis bidder ran away
-----------
bad price difficulti connect execut zero transper
-----------
recommend price inspect price total differ
-----------
warst
-----------
took 3 day respond price statu long compani claim sell 2 hr
-----------
good rate
-----------
big gap indic price rang actual post inspect
-----------
cheat hype advertis otherwis benifici custom
low_price
-----------
person experi olx auto inspect person good complet day target inspect vehicl mintu drive test drive vehicl rude
-----------
offer given oxl auto much lesser got offer 5 5 lakh howev player market offer 6 5 lakh 6 75 lakh finalis deal 6 75 lakh multipl time home inspect cancel without contact inspector late 30 45 min think would refer olx auto futur fool advertis social media realiti offer worst market
low_price
-----------
estim quotat provid buyer could arrang best price
low_price
-----------
share reason purchas


upto inspect smooth get messag mail offer price inspect instead receiv sever call enquir best price receiv promis match ultim sold anoth parti profession sent messag offer price revis
-----------
week still wait call olx would never recommend deal olx realli sorri
no_response
-----------
person profession check car complet patienc took around hour inspect
-----------
way deal start end bad end amount said day given final day
-----------
good
-----------
rate offer le
low_price
-----------
manupl repersetet given worst lowest price market
low_price
-----------
follow book four time get car inspect person never came time inspit prior book
-----------
delay respons still quit convey
no_response
-----------
quot provid despit two week
low_price
-----------
car given price explan prove file
-----------
receiv price inspect still wait bad experi
no_response
-----------
mislead messag clariti price
low_price
-----------
proper respons bother
no_response
-----------
extrem unprofession one sha